# Monte Carlo simulation on 3D Heisenberg model

This is a 3D Heisenberg model, extending from the 3D Ising (Ref. 3D Ising model).
This model allow the spin change both direction and magnitude.

In [ ]:
#----------------------------------------------------------------------#
#  Import modules
#----------------------------------------------------------------------#
from __future__ import print_function
import numpy as np
from mpi4py import MPI

#----------------------------------------------------------------------#
#   Monte Carlo simulation parameters
#----------------------------------------------------------------------#
SIZE = 10                       # MC size SIZE ^ 3
STEPS = 10000                   # MC steps

# Nearest neighbor Heisenberg interaction parameter from DFT calculations (unit: meV)
S  = 2.5
J  = [-5.802, -5.448, -0.168, 0.0]
K  = 1

#----------------------------------------------------------------------#
#   Define a function myFloat transform myList to float
#----------------------------------------------------------------------#
def myFloat(myList):
    return map(float, myList)

#----------------------------------------------------------------------#          
#  Check periodic boundary conditions                                             
#----------------------------------------------------------------------#          
def bc(i):                                                                        
    if i > SIZE-1:                                                                
        return i-SIZE                                                             
    if i < 0:                                                                     
        return i+SIZE                                                             
    else:                                                                         
        return i

#----------------------------------------------------------------------#          
#  define a normalize function                                                    
#----------------------------------------------------------------------#          
def normalize(v):                                                                 
    norm = np.linalg.norm(v)                                                      
    if norm==0:                                                                   
        return v                                                                  
    return v/norm                                                                 
                                                                                  
#----------------------------------------------------------------------#          
#   calculate average magnetization                                               
#----------------------------------------------------------------------#          
def ave_mag(system):                                                              
    totmag = np.array([0.0, 0.0, 0.0])                                            
    for i in range(SIZE):                                                         
        for j in range(SIZE):                                                     
            for k in range(SIZE):                                                 
                totmag = totmag + system[i,j,k]*(-1)**(i+j+k)  ### !!! only for G type          
                                                                                  
    return totmag/SIZE**3                                                         

#----------------------------------------------------------------------#          
#  Initial system                                                                 
#----------------------------------------------------------------------#          
def build_system():                                                               
    np.random.seed(1)                                                             
    system = np.random.ranf(size=(SIZE,SIZE,SIZE,3))                              
    for i in range(SIZE):                                                         
        for j in range(SIZE):                                                     
            for k in range(SIZE):                                                 
                for m in range(3):                                                
                    if system[i, j, k, m] >= 0.5:                                 
                        system[i, j, k, m] = system[i, j, k, m] - 1               
                system[i, j, k] = normalize(system[i, j, k])                      
    return system                                                                 
                                                                                  
#print build_system()  


#----------------------------------------------------------------------#
#  Calculate energy of single atom (upto second nearest interaction)
#  J1 - inplane nearest
#  J2 - out-of-plane nearest
#  J3 - inplane second nearest
#  J4 - out-of-plane second nearest
#----------------------------------------------------------------------#
def energy(system, X, Y, Z):
    eng_j = -1.0/2 * S**2 * np.dot(system[X, Y, Z], (
        J[0] * (system[bc(X+1), Y, Z] + system[bc(X-1), Y, Z] +
            system[X, bc(Y+1), Z] + system[X, bc(Y-1), Z]) +
        J[1] * (system[X, Y, bc(Z+1)] + system[X, Y, bc(Z-1)]) +
        J[2] * (system[bc(X+1), bc(Y+1), Z] + system[bc(X-1), bc(Y+1), Z] +
            system[bc(X+1), bc(Y-1), Z] + system[bc(X-1), bc(Y-1), Z]) +
        J[3] * (system[bc(X+1), Y, bc(Z+1)] + system[bc(X-1), Y, bc(Z+1)] +
            system[bc(X+1), Y, bc(Z-1)] + system[bc(X-1), Y, bc(Z-1)] +
            system[X, bc(Y+1), bc(Z+1)] + system[X, bc(Y-1), bc(Z+1)] +
            system[X, bc(Y+1), bc(Z-1)] + system[X, bc(Y-1), bc(Z-1)])))

    eng_h = -1.0 * S**2 * K * system[X, Y, Z, 2]**2
    eng = eng_j + eng_h
    return eng

def local_eng(system, X, Y, Z):
    loc_eng = energy(system, X, Y, Z)
        + energy(system, bc(X+1), Y, Z) + energy(system, bc(X-1), Y, Z) 
        + energy(system, X, bc(Y+1), Z) + energy(system, X, bc(Y-1), Z) 
        + energy(system, X, Y, bc(Z+1)) + energy(system, X, Y, bc(Z-1)) 
        + energy(system, bc(X+1), bc(Y+1), Z) + energy(system, bc(X-1), bc(Y+1), Z) 
        + energy(system, bc(X+1), bc(Y-1), Z) + energy(system, bc(X-1), bc(Y-1), Z) 
        + energy(system, bc(X+1), Y, bc(Z+1)) + energy(system, bc(X-1), Y, bc(Z+1)) 
        + energy(system, bc(X+1), Y, bc(Z-1)) + energy(system, bc(X-1), Y, bc(Z-1)) 
        + energy(system, X, bc(Y+1), bc(Z+1)) + energy(system, X, bc(Y-1), bc(Z+1)) 
        + energy(system, X, bc(Y+1), bc(Z-1)) + energy(system, X, bc(Y-1), bc(Z-1))
    return loc_eng

#----------------------------------------------------------------------#
#   Monte carlo loops (at temperature T)
#----------------------------------------------------------------------#

def MC_loop(T):
    filename = "spin_config_" + str(T)
    f = open(filename, 'w+')
    mag = []
    eng = []
    system = build_system()
    n_acpt = 0                          # Number of accepted MC steps
    damp = 0.1

    for step, x in enumerate(range(STEPS)):
        for X in range(0, SIZE):
            for Y in range(0, SIZE):
                for Z in range(0, SIZE):

                    local_energy = local_eng(system, X, Y, Z)
                    record_spin = system[X, Y, Z].copy()

                    ds = damp * (np.random.ranf(size=(3)) - 0.5)
                    system[X, Y, Z] = normalize(system[X, Y, Z] + ds).copy()

                    new_local_energy = local_eng(system, X, Y, Z)
                    dE = new_local_energy - local_energy
                    
                    if dE <= 0.:
                        n_acpt += 1                                               
                    elif np.exp(-1.0 * dE / T) > np.random.ranf():                
                        n_acpt += 1                                               
                    else:                                                         
                        system[X, Y, Z] = record_spin

        if step%(STEPS/100) == 0:                                                 
#            np.savetxt(filename, system.flatten(), delimiter=' ', fmt='%6.3f')   
#            print("Step ", step, file=f)                                         
            print(system, "\n", file=f)                                           
            mag = np.absolute(ave_mag(system))                                    
            print("Step ", step, ' '.join(map(str, mag)), "\n", file=f)           
    mag = np.absolute(ave_mag(system))                                            
    f.close()                                                                     
    return mag, system                                                            

#----------------------------------------------------------------------#          
#   Temperature vs. Magnetization (averaged)                                      
#   paralellize with mpi4py                                                       
#----------------------------------------------------------------------#          
comm = MPI.COMM_WORLD                                                             
                                                                                  
size = comm.Get_size()                          # Number of cores used            
rank = comm.Get_rank()                          # Rank of the core                
                                                                                  
n_temp = 1                                                                        
n_tot = n_temp * size                                                             
                                                                                  
if rank == 0:                                                                     
    temp = [0.5+i*10 for i in range(n_tot)]                                       
else:                                                                             
    temp = None                                                                   
                                                                                  
TEMP = comm.scatter(temp, root=0)                                                 
MAG = MC_loop(TEMP)[0]                                                            
                                                                                  
mag = comm.gather(MAG,root=0)                                                     
                                                                                  
if rank == 0:                                                                     
    data = np.column_stack((np.array(temp), np.array(mag)))                       
    np.savetxt('data.txt', data, delimiter=' ', fmt='%6.3f')